In [1]:
peopleDF = spark.read.parquet("people-10m.parquet")
peopleDF.show(5)

+-------+---------+----------+---------+------+-------------------+-----------+------+
|     id|firstName|middleName| lastName|gender|          birthDate|        ssn|salary|
+-------+---------+----------+---------+------+-------------------+-----------+------+
|8807768|   Darren|      Minh|Pilsworth|     M|1967-03-07 14:00:00|908-34-1027| 45983|
|8807769|  Winston|     Hiram|    Ahlin|     M|1957-07-10 13:30:00|946-32-4802| 45867|
|8807770|   Arnold|     Jason|   Durran|     M|1987-10-18 13:00:00|978-34-4842| 58623|
|8807771|    Garth|      Dino|   Eddins|     M|1998-07-16 13:00:00|992-39-5689| 33952|
|8807772|Sebastian|      Levi|    Evans|     M|1958-01-23 13:30:00|963-59-2493| 31453|
+-------+---------+----------+---------+------+-------------------+-----------+------+
only showing top 5 rows



In [7]:
name_list = [c for c in peopleDF.columns if 'Name' in c]
print(name_list)
nameDF = peopleDF[name_list]
nameDF.show(1)

nameDF.groupBy('middleName').count().show(5)

['firstName', 'middleName', 'lastName']
+---------+----------+---------+
|firstName|middleName| lastName|
+---------+----------+---------+
|   Darren|      Minh|Pilsworth|
+---------+----------+---------+
only showing top 1 row

+----------+-----+
|middleName|count|
+----------+-----+
|   Rosendo| 3956|
|   Normand| 4100|
|  Laurence| 3960|
|     Tyler| 3999|
|      Reta| 1277|
+----------+-----+
only showing top 5 rows



In [12]:
firstName = peopleDF.select('firstName').distinct()
print(firstName.count())
sortingDF = firstName.orderBy("firstName",ascending=False)
sortingDF.show(5)

5113
+---------+
|firstName|
+---------+
|    Zulma|
|   Zulema|
|     Zula|
|  Zoraida|
|     Zora|
+---------+
only showing top 5 rows



In [22]:
from pyspark.sql.functions import year,col
dordonDF = peopleDF.\
            select(year('birthDate').alias("birthYear"),"firstName").\
            filter((col("firstName") == "Donna") | (col("firstName") == "Dorothy")).\
            filter("gender == 'F'").\
            orderBy('birthYear').\
            groupBy('firstName','birthYear').\
            count().show(5)

+---------+---------+-----+
|firstName|birthYear|count|
+---------+---------+-----+
|  Dorothy|     1954|   29|
|    Donna|     1985|   27|
|    Donna|     1974|   20|
|  Dorothy|     1984|   27|
|    Donna|     1968|   27|
+---------+---------+-----+
only showing top 5 rows



In [28]:
dordonDF = peopleDF.\
            select(year('birthDate').alias("birthYear"),"firstName").\
            filter((col("firstName") == "Donna") | (col("firstName") == "Dorothy")).\
            filter("gender == 'F'").\
            filter(year("birthDate") > "1990").\
            orderBy('birthYear').\
            groupBy('firstName','birthYear').\
            count()

In [76]:
display(dordonDF)
dordonDF.show()

DataFrame[firstName: string, birthYear: int, count: bigint]

+---------+---------+-----+
|firstName|birthYear|count|
+---------+---------+-----+
|    Donna|     1996|   23|
|    Donna|     1998|   28|
|    Donna|     2000|    2|
|  Dorothy|     1992|   39|
|    Donna|     1991|   27|
|    Donna|     1999|   22|
|  Dorothy|     1998|   33|
|  Dorothy|     1996|   19|
|  Dorothy|     1993|   28|
|  Dorothy|     1994|   21|
|  Dorothy|     1995|   26|
|  Dorothy|     1999|   25|
|  Dorothy|     2000|    4|
|    Donna|     1997|   36|
|    Donna|     1994|   26|
|  Dorothy|     1991|   25|
|    Donna|     1995|   33|
|    Donna|     1992|   30|
|  Dorothy|     1997|   24|
|    Donna|     1993|   22|
+---------+---------+-----+



In [41]:
mathS_df = spark.createDataFrame([('tom',70),('bob',100),('sam',60),('lee',85),('jun',75)],['name','score'])
englishS_df = spark.createDataFrame([('tom',55),('sam',88),('kim',90),('jun',67),('choi',95)],['name','score'])


mathS_df = mathS_df.select('name',col('score').alias('math score'))
englishS_df = englishS_df.select('name',col('score').alias('english score'))


join1 = mathS_df.join(englishS_df, mathS_df.name == englishS_df.name)
join1.show()

+----+----------+----+-------------+
|name|math score|name|english score|
+----+----------+----+-------------+
| jun|        75| jun|           67|
| tom|        70| tom|           55|
| sam|        60| sam|           88|
+----+----------+----+-------------+



In [58]:
mathS_df = spark.createDataFrame([('tom',70),('bob',100),('sam',60),('lee',85),('jun',75)],['name','score'])
englishS_df = spark.createDataFrame([('tom',55),('sam',88),('kim',90),('jun',67),('choi',95)],['name','score'])


mathS_df = mathS_df.select('name',col('score').alias('math score'))
englishS_df = englishS_df.select('name',col('score').alias('english score'))

join2 = mathS_df.join(englishS_df, mathS_df.name == englishS_df.name,how='full').\
                filter(mathS_df.name.isNull())
    
join2.show()

+----+----------+----+-------------+
|name|math score|name|english score|
+----+----------+----+-------------+
|null|      null|choi|           95|
|null|      null| kim|           90|
+----+----------+----+-------------+



In [59]:
peopleDF = spark.read.parquet("people-10m.parquet")
peopleDF.show(5)

+-------+---------+----------+---------+------+-------------------+-----------+------+
|     id|firstName|middleName| lastName|gender|          birthDate|        ssn|salary|
+-------+---------+----------+---------+------+-------------------+-----------+------+
|8807768|   Darren|      Minh|Pilsworth|     M|1967-03-07 14:00:00|908-34-1027| 45983|
|8807769|  Winston|     Hiram|    Ahlin|     M|1957-07-10 13:30:00|946-32-4802| 45867|
|8807770|   Arnold|     Jason|   Durran|     M|1987-10-18 13:00:00|978-34-4842| 58623|
|8807771|    Garth|      Dino|   Eddins|     M|1998-07-16 13:00:00|992-39-5689| 33952|
|8807772|Sebastian|      Levi|    Evans|     M|1958-01-23 13:30:00|963-59-2493| 31453|
+-------+---------+----------+---------+------+-------------------+-----------+------+
only showing top 5 rows



In [71]:
import pyspark.sql.functions as fn
top10MaleFirstNameDF = peopleDF.\
                        select("firstName").\
                        filter("gender = 'M'").\
                        groupBy("firstName").\
                        agg(fn.count("firstName").alias('total')).\
                        orderBy("total",ascending = False).\
                        limit(10)

top10MaleFirstNameDF.show(5)
                                

+---------+-----+
|firstName|total|
+---------+-----+
|  Marcelo| 4197|
| Reynaldo| 4193|
|  Jeffrey| 4177|
|   Sammie| 4170|
|     Troy| 4167|
+---------+-----+
only showing top 5 rows



In [75]:
top10MaleFirstNameDF.createOrReplaceTempView("top10MaleFirstName")
resultDF = spark.sql("select * from top10MaleFirstName order by firstname")
display(resultDF)
resultDF.show()

DataFrame[firstName: string, total: bigint]

+---------+-----+
|firstName|total|
+---------+-----+
|    Darin| 4166|
|    Fritz| 4157|
|  Jeffrey| 4177|
|     Jess| 4153|
|  Marcelo| 4197|
| Reynaldo| 4193|
|   Sammie| 4170|
|    Tomas| 4161|
|     Troy| 4167|
|    Wyatt| 4159|
+---------+-----+

